# 📊 Datathon FIAP - Passos Mágicos
## Modelo Preditivo de Risco de Defasagem

Este notebook desenvolve um modelo de Machine Learning para identificar alunos em risco de defasagem educacional, utilizando os indicadores do PEDE dos anos 2022, 2023 e 2024.

**Objetivo:** Criar um modelo preditivo que identifique padrões nos indicadores que permitam prever alunos em risco de defasagem escolar.

**Autor:** Leandro Leme Crespo

---

## 1. Configuração do Ambiente

In [ ]:
# Clonar o repositório do GitHub (executar apenas no Google Colab)
!git clone https://github.com/LeandroCrespo/datathon-passos-magicos.git
print('✅ Repositório clonado com sucesso!')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl scikit-learn imbalanced-learn -q

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score, roc_auc_score,
                             roc_curve, precision_recall_curve)
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas com sucesso!')

## 2. Carregamento e Preparação dos Dados (3 Anos)

In [ ]:
# Carregar dados (do repositório GitHub)
CAMINHO_ARQUIVO = '/content/datathon-passos-magicos/data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

xlsx = pd.ExcelFile(CAMINHO_ARQUIVO)
df_2022 = pd.read_excel(xlsx, sheet_name='PEDE2022')
df_2023 = pd.read_excel(xlsx, sheet_name='PEDE2023')
df_2024 = pd.read_excel(xlsx, sheet_name='PEDE2024')

print(f'📊 Dados carregados:')
print(f'   PEDE 2022: {df_2022.shape[0]:,} alunos')
print(f'   PEDE 2023: {df_2023.shape[0]:,} alunos')
print(f'   PEDE 2024: {df_2024.shape[0]:,} alunos')
print(f'   TOTAL: {df_2022.shape[0] + df_2023.shape[0] + df_2024.shape[0]:,} registros')

In [ ]:
# Função para padronizar colunas
def padronizar_colunas(df, ano):
    df = df.copy()
    mapeamento = {}
    
    # Mapeamento específico por ano para INDE e PEDRA
    if ano == '2022':
        mapeamento['INDE 22'] = 'INDE'
        mapeamento['Pedra 22'] = 'PEDRA'
    elif ano == '2023':
        mapeamento['INDE 2023'] = 'INDE'
        mapeamento['Pedra 2023'] = 'PEDRA'
    elif ano == '2024':
        mapeamento['INDE 2024'] = 'INDE'
        mapeamento['Pedra 2024'] = 'PEDRA'
    
    # Mapeamento genérico para outras colunas
    for col in df.columns:
        col_lower = col.lower()
        if col in mapeamento:
            continue
        if col_lower == 'idade' or col_lower.startswith('idade'):
            mapeamento[col] = 'IDADE'
        elif col_lower == 'iaa':
            mapeamento[col] = 'IAA'
        elif col_lower == 'ieg':
            mapeamento[col] = 'IEG'
        elif col_lower == 'ips':
            mapeamento[col] = 'IPS'
        elif col_lower == 'ipp':
            mapeamento[col] = 'IPP'
        elif col_lower == 'ida':
            mapeamento[col] = 'IDA'
        elif col_lower == 'ipv':
            mapeamento[col] = 'IPV'
        elif col_lower == 'ian':
            mapeamento[col] = 'IAN'
        elif 'defas' in col_lower:
            mapeamento[col] = 'DEFASAGEM'
    
    df = df.rename(columns=mapeamento)
    df = df.loc[:, ~df.columns.duplicated()]  # Remover colunas duplicadas
    df['ANO'] = int(ano)
    return df

# Padronizar os 3 anos
df_2022_pad = padronizar_colunas(df_2022, '2022')
df_2023_pad = padronizar_colunas(df_2023, '2023')
df_2024_pad = padronizar_colunas(df_2024, '2024')

print('✅ Colunas padronizadas com sucesso!')

In [ ]:
# Selecionar colunas comuns
colunas = ['ANO', 'INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM']

def selecionar_colunas(df, colunas):
    return df[[c for c in colunas if c in df.columns]].copy()

df_2022_sel = selecionar_colunas(df_2022_pad, colunas)
df_2023_sel = selecionar_colunas(df_2023_pad, colunas)
df_2024_sel = selecionar_colunas(df_2024_pad, colunas)

# Combinar os 3 anos
df_unificado = pd.concat([df_2022_sel, df_2023_sel, df_2024_sel], ignore_index=True)

# Converter para numérico
colunas_numericas = ['INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM']
for col in colunas_numericas:
    if col in df_unificado.columns:
        df_unificado[col] = pd.to_numeric(df_unificado[col], errors='coerce')

print(f'✅ DataFrame unificado: {len(df_unificado):,} registros')
print(f'\nDistribuição por ano:')
print(df_unificado['ANO'].value_counts().sort_index())

## 3. Definição da Variável Alvo (Risco de Defasagem)

A coluna **DEFASAGEM** representa a diferença entre a fase atual do aluno e a fase ideal para sua idade:
- **Valores negativos**: Aluno está adiantado (fase acima do esperado)
- **Zero**: Aluno está na fase correta para sua idade
- **Valores positivos**: Aluno está atrasado/defasado (fase abaixo do esperado)

**Critério de risco:** Consideramos em risco os alunos com DEFASAGEM > 0 (atrasados em relação à fase ideal).

In [ ]:
# Definir variável alvo: aluno em risco de defasagem
# Critério: defasagem > 0 (aluno está atrasado em relação à fase ideal)

df_unificado['RISCO_DEFASAGEM'] = (df_unificado['DEFASAGEM'] > 0).astype(int)

print('📊 Distribuição da variável alvo (RISCO_DEFASAGEM):')
print(df_unificado['RISCO_DEFASAGEM'].value_counts())
print(f'\nPercentual em risco: {df_unificado["RISCO_DEFASAGEM"].mean()*100:.1f}%')

In [ ]:
# Visualizar distribuição
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Distribuição da variável alvo
cores = ['#2ecc71', '#e74c3c']
labels = ['Sem risco (0)', 'Com risco (1)']
valores = df_unificado['RISCO_DEFASAGEM'].value_counts().sort_index()
axes[0].pie(valores, labels=labels, colors=cores, autopct='%1.1f%%', startangle=90, explode=[0, 0.1])
axes[0].set_title('Distribuição da Variável Alvo', fontweight='bold', fontsize=14)

# Gráfico 2: Risco por ano
risco_ano = df_unificado.groupby('ANO')['RISCO_DEFASAGEM'].mean() * 100
bars = axes[1].bar(risco_ano.index.astype(str), risco_ano.values, color='#e74c3c', edgecolor='black')
axes[1].set_title('Percentual de Alunos em Risco por Ano', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Ano')
axes[1].set_ylabel('% em Risco')
for bar, val in zip(bars, risco_ano.values):
    axes[1].annotate(f'{val:.1f}%', (bar.get_x() + bar.get_width()/2, bar.get_height()),
                     textcoords='offset points', xytext=(0, 5), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('distribuicao_risco.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Engineering

In [ ]:
# Selecionar features para o modelo
features = ['IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP', 'IPV', 'INDE']
features_existentes = [f for f in features if f in df_unificado.columns]

print(f'Features disponíveis: {features_existentes}')

# Preparar dados para o modelo
df_modelo = df_unificado[features_existentes + ['RISCO_DEFASAGEM']].dropna()
print(f'\nRegistros após remover nulos: {len(df_modelo):,}')

X = df_modelo[features_existentes].copy()
y = df_modelo['RISCO_DEFASAGEM']

# Criar feature derivada: média dos indicadores
X['MEDIA_INDICADORES'] = X[features_existentes].mean(axis=1)

print(f'\nFeatures finais: {list(X.columns)}')
print(f'Total de features: {len(X.columns)}')

## 5. Separação dos Dados em Treino e Teste

In [ ]:
# Dividir em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Manter proporção das classes
)

print(f'📊 Divisão dos dados:')
print(f'   Treino: {len(X_train):,} registros ({len(X_train)/len(X)*100:.0f}%)')
print(f'   Teste: {len(X_test):,} registros ({len(X_test)/len(X)*100:.0f}%)')

print(f'\nDistribuição da classe alvo no treino:')
print(f'   Sem risco: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)')
print(f'   Com risco: {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.1f}%)')

## 6. Balanceamento com SMOTE

Como temos um desbalanceamento significativo entre as classes (poucos alunos em risco), utilizamos a técnica **SMOTE** (Synthetic Minority Over-sampling Technique) para criar exemplos sintéticos da classe minoritária.

In [ ]:
# Aplicar SMOTE para balancear as classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f'📊 Após aplicar SMOTE:')
print(f'   Treino original: {len(X_train):,} registros')
print(f'   Treino balanceado: {len(X_train_smote):,} registros')

print(f'\nDistribuição após SMOTE:')
print(f'   Sem risco: {(y_train_smote == 0).sum():,} ({(y_train_smote == 0).mean()*100:.1f}%)')
print(f'   Com risco: {(y_train_smote == 1).sum():,} ({(y_train_smote == 1).mean()*100:.1f}%)')

## 7. Normalização dos Dados

In [ ]:
# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

print('✅ Dados normalizados com sucesso!')

## 8. Treinamento do Modelo (Random Forest)

In [ ]:
# Treinar modelo Random Forest
modelo = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

modelo.fit(X_train_scaled, y_train_smote)

print('✅ Modelo Random Forest treinado com sucesso!')

## 9. Avaliação do Modelo

In [ ]:
# Predições com threshold ajustado
# Usamos threshold = 0.3 para priorizar o Recall (identificar mais alunos em risco)
y_proba = modelo.predict_proba(X_test_scaled)[:, 1]
threshold = 0.3
y_pred = (y_proba >= threshold).astype(int)

# Métricas
print('='*60)
print(f'RESULTADOS DO MODELO (threshold={threshold})')
print('='*60)
print(f'   Acurácia:  {accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'   Precisão:  {precision_score(y_test, y_pred)*100:.2f}%')
print(f'   Recall:    {recall_score(y_test, y_pred)*100:.2f}%')
print(f'   F1-Score:  {f1_score(y_test, y_pred)*100:.2f}%')
print(f'   AUC-ROC:   {roc_auc_score(y_test, y_proba)*100:.2f}%')

In [ ]:
# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Sem Risco', 'Com Risco'],
            yticklabels=['Sem Risco', 'Com Risco'])
ax.set_xlabel('Predito', fontsize=12)
ax.set_ylabel('Real', fontsize=12)
ax.set_title('Matriz de Confusão', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('matriz_confusao.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nInterpretação:')
print(f'   Verdadeiros Negativos (TN): {cm[0,0]} - Alunos sem risco corretamente identificados')
print(f'   Falsos Positivos (FP): {cm[0,1]} - Alunos sem risco incorretamente classificados como em risco')
print(f'   Falsos Negativos (FN): {cm[1,0]} - Alunos em risco não identificados')
print(f'   Verdadeiros Positivos (TP): {cm[1,1]} - Alunos em risco corretamente identificados')

In [ ]:
# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, color='#3498db', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random classifier')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Taxa de Falsos Positivos', fontsize=12)
ax.set_ylabel('Taxa de Verdadeiros Positivos', fontsize=12)
ax.set_title('Curva ROC', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('curva_roc.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Importância das Features

In [ ]:
# Importância das features
feature_names = list(X.columns)
importances = modelo.feature_importances_

# Ordenar por importância
indices = np.argsort(importances)[::-1]

print('📊 Importância das Features:')
print('='*40)
for i, idx in enumerate(indices):
    print(f'{i+1}. {feature_names[idx]}: {importances[idx]*100:.1f}%')

# Gráfico
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.Blues(np.linspace(0.4, 0.8, len(feature_names)))
bars = ax.barh([feature_names[i] for i in indices], [importances[i] for i in indices], color=colors[::-1])
ax.set_xlabel('Importância', fontsize=12)
ax.set_title('Importância das Features no Modelo', fontsize=14, fontweight='bold')

# Adicionar valores nas barras
for bar, imp in zip(bars, [importances[i] for i in indices]):
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
            f'{imp*100:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Salvamento do Modelo

In [ ]:
import pickle

# Salvar modelo e artefatos
with open('modelo_risco_defasagem.pkl', 'wb') as f:
    pickle.dump(modelo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('features.txt', 'w') as f:
    f.write('\n'.join(feature_names))

with open('threshold.txt', 'w') as f:
    f.write(str(threshold))

print('✅ Modelo e artefatos salvos com sucesso!')
print('   - modelo_risco_defasagem.pkl')
print('   - scaler.pkl')
print('   - features.txt')
print('   - threshold.txt')

## 12. Conclusões

### Resultados do Modelo

O modelo Random Forest treinado com dados dos 3 anos (2022-2024) apresentou os seguintes resultados:

| Métrica | Valor |
|---------|-------|
| **Acurácia** | 85.64% |
| **Recall** | 65.38% |
| **AUC-ROC** | 87.75% |
| **F1-Score** | 37.36% |

### Features Mais Importantes

1. **IAN (Adequação ao Nível)**: 29.9% - Principal preditor de risco
2. **IPS (Psicossocial)**: 11.8%
3. **MEDIA_INDICADORES**: 10.0%
4. **IAA (Autoavaliação)**: 10.0%
5. **IPV (Ponto de Virada)**: 8.3%

### Interpretação

- O **AUC-ROC de 87.75%** indica excelente capacidade de discriminação entre alunos em risco e sem risco.
- O **Recall de 65.38%** significa que o modelo identifica aproximadamente 2 em cada 3 alunos que realmente estão em risco.
- O **IAN** é o indicador mais importante, confirmando que a adequação ao nível é o principal fator de risco.

### Recomendações

1. Utilizar o modelo como ferramenta de triagem para identificar alunos que necessitam de atenção especial.
2. Priorizar intervenções em alunos com IAN baixo.
3. Monitorar indicadores psicossociais (IPS) e de autoavaliação (IAA) como sinais de alerta.